In [1]:
import glob
import pandas as pd
from collections import Counter
import numpy as np
import time
import datetime

In [2]:
#Read artists master data that will be joined to concatenated list of all social data
artists = pd.read_csv('master_artists_list.csv')

In [3]:
artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube', 'musicbrainz'],
      dtype='object')

In [4]:
artists['artist_name']

0                    Shakira
1                    Beyoncé
2             Jennifer Lopez
3                Miley Cyrus
4          Justin Timberlake
5                      Adele
6                Chris Brown
7             Britney Spears
8           Enrique Iglesias
9                    Pitbull
10                    Maluma
11                 Nicky Jam
12                      Pink
13             Avril Lavigne
14             Calvin Harris
15              Romeo Santos
16                      Zayn
17               Alicia Keys
18                Snoop Dogg
19              Harry Styles
20                     Usher
21              Ricky Martin
22              Mariah Carey
23              Prince Royce
24        Christina Aguilera
25                       Sia
26             Martin Garrix
27                Little Mix
28           Camilla Cabello
29                    Thalia
               ...          
100             Diego Torres
101               Kane Brown
102                   Hozier
103           

In [5]:
#Read all social data files in
files = ['twitter_features.csv', 'youtube_to_combine.csv', 'facebook_7yrs.csv']

In [6]:
df_list = [pd.read_csv(file) for file in files]
df_list.append(pd.read_pickle('instagram_7yrs_cleaned.pkl'))

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [7]:
sum = 0
for d in df_list:
    print("num columns", len(d.columns))
    sum+=d.shape[0]
    print(d.shape)
    print(d.columns)

print("number of rows in 4 dataframes: ", sum)

num columns 22
(305308, 22)
Index(['Unnamed: 0', 'Twitter_Handle', 'count_of_followers', 'created',
       'description', 'crowdtangle_score', 'description_length',
       'hashtag_count', 'type_link', 'type_photo', 'type_text', 'type_video',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'time_since_last_post', 'within_week_release',
       'within_month_release', 'social_engagement_score', 'last_7_days',
       'last_30_days'],
      dtype='object')
num columns 22
(20441, 22)
Index(['artist_name', 'count_of_followers', 'created', 'created_12am-6am',
       'created_12pm-6pm', 'created_6am-12pm', 'created_6pm-12am',
       'crowdtangle_score', 'description', 'description_length',
       'hashtag_count', 'social_engagement_score', 'time_of_day',
       'time_since_last_post', 'type_link', 'type_photo', 'type_text',
       'type_video', 'within_month_release', 'within_week_release',
       'last_7_days', 'last_30_days'],
      dtype='object

In [8]:
df_list[1].columns

Index(['artist_name', 'count_of_followers', 'created', 'created_12am-6am',
       'created_12pm-6pm', 'created_6am-12pm', 'created_6pm-12am',
       'crowdtangle_score', 'description', 'description_length',
       'hashtag_count', 'social_engagement_score', 'time_of_day',
       'time_since_last_post', 'type_link', 'type_photo', 'type_text',
       'type_video', 'within_month_release', 'within_week_release',
       'last_7_days', 'last_30_days'],
      dtype='object')

In [9]:
#Temporary fixes for youtube
df_list[1].drop(columns='time_of_day', inplace=True)
df_list[1].reset_index(inplace=True)
df_list[1].rename(columns = {'index': 'Unnamed: 0'}, inplace=True)

In [10]:
#Renaming columns that were not aligned
df_list[2].rename(columns = {'fb_handle_artist_name': 'artist_name'}, inplace=True) #Facebook
df_list[0].rename(columns = {'Twitter_Handle': 'artist_name'}, inplace=True) #Twitter

In [11]:
#Adding Dummy columns to indicate which dataset it is
df_list[0]['Facebook'] = 0
df_list[0]['Instagram'] = 0
df_list[0]['Twitter'] = 1
df_list[0]['Youtube'] = 0

df_list[1]['Facebook'] = 0
df_list[1]['Instagram'] = 0
df_list[1]['Twitter'] = 0
df_list[1]['Youtube'] = 1

df_list[2]['Facebook'] = 1
df_list[2]['Instagram'] = 0
df_list[2]['Twitter'] = 0
df_list[2]['Youtube'] = 0

df_list[3]['Facebook'] = 0
df_list[3]['Instagram'] = 1
df_list[3]['Twitter'] = 0
df_list[3]['Youtube'] = 0

In [12]:
fb = df_list[2].merge(artists, left_on='artist_name', right_on='facebook')
instagram = df_list[3].merge(artists, left_on='artist_name', right_on='instagram')
twitter = df_list[0].merge(artists, left_on='artist_name', right_on='twitter')
yt = df_list[1].merge(artists, left_on='artist_name', right_on='youtube')
socials = [fb, instagram, twitter, yt]

In [13]:
for s in socials:
    print(s.shape)

(297603, 45)
(304031, 44)
(305308, 45)
(20441, 45)


###### Investigation Start

In [14]:
fb_check = df_list[2].merge(artists, how='outer', left_on='artist_name', right_on='facebook')


In [15]:
fb_check.shape

(297603, 45)

In [16]:
#Delta between full outer join and inner join
fb_check.shape[0] - fb.shape[0]

0

In [17]:
fb_diff = pd.concat([fb, fb_check]).drop_duplicates(keep=False)

In [18]:
fb_diff.head()

,Unnamed: 0,artist_name_x,created,description,crowdtangle_score,count_of_followers,type_link,type_photo,type_text,type_video,...,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube,musicbrainz


In [19]:
#Differences match
fb_diff.shape

(0, 45)

In [20]:
fb_diff2 = fb_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]

In [21]:
#NOTE: THIS is what was missing originally.  Will be blank if re-run
fb_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


##### Instagram Check

In [22]:
in_check = df_list[3].merge(artists, how='outer', left_on='artist_name', right_on='instagram')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - instagram.shape[0])

in_diff = pd.concat([instagram, in_check]).drop_duplicates(keep=False)

(304031, 44)
0


In [23]:
in_diff.head()

,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,social_engagement_score,type_photo,type_video,...,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube,musicbrainz


In [24]:
#Differences match
in_diff.shape

(0, 44)

In [25]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,


###### investigation stop

In [26]:
df = pd.concat(socials)
df.drop('Unnamed: 0',axis=1,inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [27]:
df.sample(10)

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,time_since_last_post,twitter,type_link,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube
28771,0,0,1,0,34,ChrisYoungMusic,Chris Young,1845782.0,2019-02-08 22:44:11,0,...,1 days 04:25:30.000000000,ChrisYoungMusic,1,0,0,0,0,0,17,Chris Young
51143,0,0,1,0,15,GraceVanderWaal,Grace VanderWaal,230137.0,2016-12-02 05:08:17,1,...,0 days 06:47:42.000000000,GraceVanderWaal,0,0,0,1,0,0,3,Oh Never Mind It’s Just Me - Grace VanderWaal
249388,1,0,0,0,30,princeroyce,Prince Royce,24877014.0,2014-07-14 15:27:11,0,...,0 days 17:13:49.000000000,PrinceRoyce,1,0,0,0,0,0,11,Prince Royce
216024,0,1,0,0,43,richforever,Rick Ross,8653568.0,2018-11-24 19:33:36,0,...,0 days 02:19:58,RickRoss,0,1,0,0,0,0,19,Rick Ross
250352,1,0,0,0,30,princeroyce,Prince Royce,33436832.0,2016-12-05 18:51:50,0,...,0 days 00:54:56.000000000,PrinceRoyce,1,0,0,0,0,0,11,Prince Royce
114743,0,0,1,0,41,Residente,Residente,5797713.0,2018-12-20 19:49:03,0,...,0 days 02:38:31.000000000,Residente,0,1,0,0,0,0,15,Residente
65393,1,0,0,0,37,NataliaJimenezOficial,Natalia Jimenez,NaN,2012-07-28 23:53:28,0,...,0 days 06:09:59.000000000,NataliaJimenez,0,0,1,0,0,0,19,Natalia Jimenez
166640,0,1,0,0,25,meghan_trainor,Meghan Trainor,6526124.0,2016-08-29 18:05:26,0,...,4 days 01:44:17,Meghan_Trainor,0,1,0,0,0,0,10,Meghan Trainor
86519,1,0,0,0,53,PearlJam,Pearl Jam,10479111.0,2016-12-27 19:49:00,0,...,0 days 23:36:00.000000000,PearlJam,0,1,0,0,0,0,29,Pearl Jam
84996,0,1,0,0,28,farrukoofficial,Farruko,6280333.0,2016-11-11 12:38:28,0,...,0 days 09:36:25,FarrukoOfficial,0,1,0,0,0,0,10,Farruko


In [28]:
print(df.shape)
print("number of rows lost: ", sum - df.shape[0], ", ", (sum - df.shape[0])/sum)

(927383, 44)
number of rows lost:  0 ,  0.0


In [29]:
df.head(3).T

,0,1,2
Facebook,1,1,1
Instagram,0,0,0
Twitter,0,0,0
Youtube,0,0,0
age,26,26,26
artist_name_x,21Savage,21Savage,21Savage
artist_name_y,21 Savage,21 Savage,21 Savage
count_of_followers,NaN,NaN,NaN
created,2015-05-18 19:24:23,2015-05-25 16:00:00,2015-05-25 22:26:12
created_12am-6am,0,0,0


In [30]:
# df.to_csv('Combined_Social_Data.csv')

In [31]:
# Filtering outliers in FB data
df = df[~((df['Facebook'] == 1) & (df['social_engagement_score'] > 1))]

In [32]:
# Get the social platform name for each dataframe row
def getSocialPlatform(x):
    if x['Facebook'] == 1:
        return 'Facebook'
    elif x['Instagram'] == 1:
        return 'Instagram'
    elif x['Twitter'] == 1:
        return 'Twitter'
    else:
        return 'Youtube'

In [33]:
# Create social platform column
df['social_platform'] = df.apply(lambda x: getSocialPlatform(x), axis = 1)

In [34]:
# Get the mean & standard deviation of social engagement scores per platform and store the results in 3 dictionaries
dict_platform_mean = df.groupby('social_platform')['social_engagement_score'].mean().to_dict()
dict_platform_std = df.groupby('social_platform')['social_engagement_score'].std().to_dict()

In [35]:
# Normalize the social engagement scores
df['normalized_social_engagement_score'] = df.apply(lambda x: (x['social_engagement_score'] - dict_platform_mean[x['social_platform']]) / dict_platform_std[x['social_platform']],axis=1)

In [36]:
# Reading music Brainz data and removing unwanted column
df_musicbrainz = pd.read_csv("musicbrainz_data.csv")
df_musicbrainz = df_musicbrainz.drop("Unnamed: 0",axis=1)

In [37]:
# Converting the type of release_date to datettime for date comparisons
df_musicbrainz['release_date'] = pd.to_datetime(df_musicbrainz['release_date'])

In [38]:
# Filter the release dates before Jan 1st 2006
df_musicbrainz = df_musicbrainz[df_musicbrainz['release_date'] > datetime.date(2006,1,1)]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [39]:
# Convert master data 'created' column to datetime for date comparisons
df['created'] = pd.to_datetime(df['created'])

In [40]:
artist_dict = df_musicbrainz.groupby('artist_name')['release_date'].apply(list).to_dict()

In [41]:
# Function to find days between 2 datetimes
from datetime import datetime
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [42]:
# Function to find the minimum difference of days w.r.t. all release dates of an artist
def find_min_diff(artist_name,created):
    lst = artist_dict[artist_name]
    min_days = min([days_between(x.strftime('%Y-%m-%d'),created.strftime('%Y-%m-%d')) for x in lst])
    return min_days

In [43]:
# Calculating the number of days to track release
t1 = time.time()
df['num_days_to_track_release'] = df.apply(lambda x: find_min_diff(x['artist_name_y'],x['created']),axis=1)
print("time taken",(time.time() - t1))

time taken 1031.1659190654755


In [44]:
df.head()

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube,social_platform,normalized_social_engagement_score,num_days_to_track_release
0,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-18 19:24:23,0,...,0,1,0,0,0,5,21 Savage,Facebook,NaN,7
1,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,0
2,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 22:26:12,0,...,0,0,0,0,0,5,21 Savage,Facebook,NaN,0
3,1,0,0,0,26,21Savage,21 Savage,NaN,2015-08-14 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,43
4,1,0,0,0,26,21Savage,21 Savage,NaN,2015-09-05 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,65


In [45]:
# Saving the dataframe to pickle file
df.to_pickle("Combined_Social_Data_Pickle.pkl")

In [46]:
df.shape

(927351, 47)

In [47]:
df.head()

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube,social_platform,normalized_social_engagement_score,num_days_to_track_release
0,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-18 19:24:23,0,...,0,1,0,0,0,5,21 Savage,Facebook,NaN,7
1,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,0
2,1,0,0,0,26,21Savage,21 Savage,NaN,2015-05-25 22:26:12,0,...,0,0,0,0,0,5,21 Savage,Facebook,NaN,0
3,1,0,0,0,26,21Savage,21 Savage,NaN,2015-08-14 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,43
4,1,0,0,0,26,21Savage,21 Savage,NaN,2015-09-05 16:00:00,0,...,1,0,0,0,0,5,21 Savage,Facebook,NaN,65
